In [71]:
import pandas as pd
import numpy as np
import os
import random
import heapq
import time
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

In [72]:
# Set Working Directory
os.getcwd()
#path = "C:/Users/Dennis/OneDrive/Dokumente/03_Master BAOR/05_Kurse/01_Business Analytics/04_Data Analytics Challenge"
#os.chdir(path)

%cd "C:\Users\Dennis\OneDrive\Dokumente\03_Master BAOR\05_Kurse\01_Business Analytics\04_Data Analytics Challenge"
ccdata = pd.read_csv('creditcard.csv')
ccdata = ccdata.iloc[:100000, :]
ccdata.Class.value_counts()

C:\Users\Dennis\OneDrive\Dokumente\03_Master BAOR\05_Kurse\01_Business Analytics\04_Data Analytics Challenge


0    99777
1      223
Name: Class, dtype: int64

In [73]:
# Drop feature 'Time'
ccdata.drop('Time', axis=1, inplace=True)

# Scaling feature 'Amount'
standard_scaler = StandardScaler()
ccdata['Amount'] = standard_scaler.fit_transform(ccdata[['Amount']])
ccdata.head(3)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.195217,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.360028,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.060755,0


In [74]:
samples = ccdata

time_start = time.time()
wrg = 0

# Create feature and target DF
samples_X = samples.iloc[:, 0:-1] #features
samples_y = samples.iloc[:,-1] #target variable

# Split it into minority set
Minority_sample = samples[samples['Class'] == 1] #minority set
Minority_sample_X = Minority_sample.iloc[:, 0:-1] #features of minority set as DF

# Create an array with only minority sample features and one with all sample features
Minority_X = np.array(Minority_sample_X) #features of Min set as Array
All_X = np.array(samples_X) #features of all samples

n1 = All_X.shape[0]-2*Minority_X.shape[0] #Anzahl aller samples - 2 * Anzahl Min samples #284807 - 2* 492 = 283823
print(n1)

99554


In [76]:
# Create distance matrix with cdist()-method
distance_matrix = pd.DataFrame(cdist(Minority_X, All_X), index=Minority_sample.index, columns=samples.index)
distance_matrix = np.where(distance_matrix == 0, 999999, distance_matrix).tolist()
distance_df = pd.DataFrame(distance_matrix, index=Minority_sample.index, columns=samples.index)

# View matrices
#display(pd.DataFrame(dis_matrix, index=Minority_sample.index, columns=samples.index).head())
display(distance_df.head())

# Check if all elements of the matrices are the same
#np.all(distance_matrix == dis_matrix)

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
541,10.357179,10.132816,11.810625,11.019541,10.787340,9.826474,10.778894,12.440274,11.515668,10.025869,...,10.397763,10.846131,10.253685,9.539368,11.390775,10.356847,11.223970,10.761188,11.489017,19.864519
623,6.295359,8.051002,7.445328,7.824044,7.241864,7.577328,8.170705,9.235025,8.763367,7.610267,...,7.793118,8.173958,8.140320,9.010206,7.838010,8.632326,7.777869,7.234145,9.222775,18.835665
4920,10.348284,11.042656,11.514605,11.031855,10.861480,10.398336,10.597156,11.387326,11.343910,10.801690,...,10.759948,11.420117,10.437539,12.382556,12.001562,10.882629,11.101812,12.350880,11.397324,19.031912
6108,22.465365,22.944860,23.094584,22.888017,22.663422,22.627415,22.757774,22.808693,23.046773,22.815832,...,22.790128,23.061692,22.810080,23.621025,22.975417,22.874139,22.896752,23.371752,23.105006,27.352589
6329,15.765642,14.573642,16.365489,15.796179,15.695163,14.716857,14.827144,16.177461,15.756975,14.732464,...,14.886932,15.965701,14.939201,10.038728,16.167578,15.893429,16.017108,16.900090,15.584456,23.280536


In [193]:
# Create empty matrix with n min instances as rows and k columns
nb_idx_matrix = np.zeros((Minority_X.shape[0], 5))

# Fill matrix with indices of k nearest neighbors for each min instance
for i in range(Minority_X.shape[0]): #für alle Min instances
    nb_idx_matrix[i] = list(map(distance_matrix[i].index, heapq.nsmallest(5, distance_matrix[i]))) #index des nächsten Nachbarn in Zeile der aktuellen Min instance

# Edit indices to integers (nested list comprehension)
nb_idx_matrix = np.array([[int(neighbor) for neighbor in neighbors] for neighbors in nb_idx_matrix])

array([[53794, 79835, 26802, 52584, 27627],
       [85890, 16842, 36083, 30868, 39135],
       [52398, 59539, 69980,  9643, 87354],
       ...,
       [86155, 59539, 46998, 57615, 52466],
       [27627, 68522, 79835, 23308, 69498],
       [79883, 23422, 43624, 53794,   472]])

In [104]:
# Create empty matrix with n min instances as rows and k columns
nb_y_matrix = np.zeros((Minority_X.shape[0], 5))

# Fill matrix with target value of k nearest neighbors for each min instance
for i, row in enumerate(nb_idx_matrix):
    for j, element in enumerate(row):
        nb_y_matrix[i,j] = samples_y[element]

nb_y_matrix

array([[1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1.],
       ...,
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0.]])

In [130]:
# Set values of maj instance neighbors to NAN
nb_y_matrix = np.where(nb_y_matrix==0, np.nan, nb_y_matrix)

# multiply both matricies to get only the indices of the neighbors from min class
q_nb_matrix = nb_y_matrix * nb_idx_matrix
q_nb_matrix

array([[53794., 79835., 26802., 52584., 27627.],
       [   nan,    nan,    nan,    nan,    nan],
       [   nan, 59539., 69980.,    nan, 87354.],
       ...,
       [86155., 59539., 46998., 57615., 52466.],
       [27627., 68522., 79835., 23308., 69498.],
       [79883., 23422., 43624., 53794.,    nan]])

In [213]:
# Select only min instances with a min instance as nearest neighbor
q_min_idx = []
for index in range(len(q_nb_matrix)):
    if (np.isnan(q_nb_matrix[index][0]) == False):
        q_min_idx.append(index)

len(q_min_idx)
#q_min_idx

191

In [201]:
q_nb_matrix[q_min_idx]

array([[53794., 79835., 26802., 52584., 27627.],
       [ 6882.,  8842.,  8845., 42007., 81186.],
       [40525.,    nan, 18466.,    nan, 18472.],
       [ 6334.,  6719.,  6336.,  8312.,  6717.],
       [ 6331.,  6719.,  6338.,  6717.,  8312.],
       [ 8312.,  6338.,  6331., 18773.,  6334.],
       [ 8312.,  6336.,  6334., 18773.,  6641.],
       [ 6446.,  6774.,  6734.,  6529.,  6472.],
       [ 6427.,  6734.,  6774.,  6472.,  6609.],
       [ 6446.,  6427.,  6609.,  6529., 79835.],
       [ 6609.,  6820.,  6870.,  6427.,  6446.],
       [ 6529.,  6870.,  6820.,  6446.,  6427.],
       [ 8617.,  6338.,  8312.,  6336.,  6334.],
       [ 6719.,  6903.,  6899.,  6334.,  6331.],
       [ 6717.,  6334.,  6331.,  6903.,  6899.],
       [ 6774.,  6427.,  6446.,  6870.,  6820.],
       [ 6734.,  6427.,  6446.,  6870.,  6820.],
       [ 6870.,  6529.,  6609.,  6734.,  6774.],
       [ 6820.,  6529.,  6609.,  6734.,  6774.],
       [ 8842.,  8845.,  6108., 88897., 42007.],
       [ 6903.,  671

In [205]:
# Select only the neighbors which are closer than the closest maj instance
q_nbs_dict = {}

for index, neighbors in enumerate(q_nb_matrix):
    if (np.isnan(q_nb_matrix[index]).any()):
        q_nbs = []
        for neighbor in neighbors:
            if np.isnan(neighbor):
                break
            else:
                q_nbs.append(int(neighbor))
        if (len(q_nbs) > 0): q_nbs_dict[index] = q_nbs
    else:
        q_nbs_dict[index] = [int(element) for element in neighbors]

q_nbs_dict

{0: [53794, 79835, 26802, 52584, 27627],
 3: [6882, 8842, 8845, 42007, 81186],
 4: [40525],
 5: [6334, 6719, 6336, 8312, 6717],
 6: [6331, 6719, 6338, 6717, 8312],
 7: [8312, 6338, 6331, 18773, 6334],
 8: [8312, 6336, 6334, 18773, 6641],
 9: [6446, 6774, 6734, 6529, 6472],
 10: [6427, 6734, 6774, 6472, 6609],
 11: [6446, 6427, 6609, 6529, 79835],
 12: [6609, 6820, 6870, 6427, 6446],
 13: [6529, 6870, 6820, 6446, 6427],
 14: [8617, 6338, 8312, 6336, 6334],
 15: [6719, 6903, 6899, 6334, 6331],
 16: [6717, 6334, 6331, 6903, 6899],
 17: [6774, 6427, 6446, 6870, 6820],
 18: [6734, 6427, 6446, 6870, 6820],
 19: [6870, 6529, 6609, 6734, 6774],
 20: [6820, 6529, 6609, 6734, 6774],
 21: [8842, 8845, 6108, 88897, 42007],
 22: [6903, 6717, 6719, 6334, 6331],
 23: [6899, 6717, 6719, 6334, 6331],
 24: [89190, 95597, 44001],
 25: [8615, 9035, 9252, 9487, 9509],
 26: [6338, 6336, 8617, 6334, 6331],
 27: [9179],
 28: [8296, 9252, 9035, 9487, 9509],
 29: [6641, 8312, 6338, 6336, 6334],
 30: [8845, 6882

In [290]:
nn = random.randint(0, len(q_nbs_dict[q_min_idx[3]]) - 1)
print(nn)
print(q_nbs_dict[q_min_idx[3]])
q_nbs_dict[q_min_idx[3]][nn]

#q_min_idx
#random.randint(0, len(q_nbs_dict[q_min_idx[3]]) - 1)

0
[6334, 6719, 6336, 8312, 6717]


6334

In [300]:
All_X[6334]
samples.iloc[6717]['Class']
Minority_X[q_min_idx[3]]

array([ 8.43036490e-03,  4.13783683e+00, -6.24069657e+00,  6.67573216e+00,
        7.68307025e-01, -3.35305955e+00, -1.63173467e+00,  1.54612448e-01,
       -2.79589246e+00, -6.18789063e+00,  5.66439471e+00, -9.85448482e+00,
       -3.06166658e-01, -1.06911962e+01, -6.38498193e-01, -2.04197379e+00,
       -1.12905588e+00,  1.16452521e-01, -1.93466574e+00,  4.88378221e-01,
        3.64514210e-01, -6.08057134e-01, -5.39527942e-01,  1.28939983e-01,
        1.48848121e+00,  5.07962678e-01,  7.35821636e-01,  5.13573741e-01,
       -3.66414466e-01])

In [321]:
len(q_min_idx)
len(q_min_idx)*n
range(n)
#np.arange(0, n)
q_nbs_dict[q_min_idx[3]]

[6334, 6719, 6336, 8312, 6717]

In [368]:
len(q_nbs_dict)
len(q_min_idx)
len(q_nbs_dict[q_min_idx[190]])
#print(q_min_idx[:5])
#print(q_nbs_dict[0])
q_nbs_dict[q_min_idx[0]]

for i in q_min_idx[:5]: print(q_nbs_dict[i])

[53794, 79835, 26802, 52584, 27627]
[6882, 8842, 8845, 42007, 81186]
[40525]
[6334, 6719, 6336, 8312, 6717]
[6331, 6719, 6338, 6717, 8312]


In [378]:
# Algorithm 2: Adaptive neighbor instances selection
n = 10 #number of synthetic new instances per qualified Min instance
k = 5
synthetic = np.zeros((len(q_min_idx)*n, Minority_X.shape[1]), dtype=float) #Matrix(qualified Min instances * n x Number of Features)

# Algorithm 3: Procedure of ASN-SMOTE (Create new synthetic minority samples)
for min in q_min_idx:
    for new_sample in range(n):
        nn = random.randint(0, len(q_nbs_dict[min]) - 1) #Zufälliges Auswählen eines besten Nachbarn aus dem Set der qualifizierten Nachbarn
        print(q_nbs_dict[min][nn])
        dif = All_X[q_nbs_dict[min][nn]] - Minority_X[min] #Differenz des zufällig ausgewählten Nachbarn und der Min instance
        print(dif)
        gap = random.random() #Wert zw. 0 und 1
        synthetic[new_sample] = Minority_X[min] + gap * dif #Generierung einer neuen Min instance

52584
[ 0.79298301  0.35650044  0.10625141 -1.933805   -0.47865719  0.40964829
  0.47765679 -1.66682362  1.20788318  0.01647522  0.23621505 -0.62162197
 -0.32353941 -0.16284647  0.10958963 -1.76715625 -2.41859072 -0.91999218
  0.74316459  0.04810799  0.79063906  0.13787501  0.44746479 -0.17050227
 -0.14112164 -0.54695437 -0.2803891  -0.06504322  0.00377898]
79835
[ 2.65261733  0.06324145 -1.16747883 -0.18588146  0.06045879  0.27452335
  0.53542864 -0.84297619  0.42604761 -0.30442644  0.35270434 -1.06978191
 -0.33696181 -1.58681354 -0.45646357 -0.35453359 -1.5947015  -0.60250077
 -0.64815409  0.19986224 -0.21746355 -0.54823356  0.27751503 -0.64945395
  0.68780897 -0.11976008  0.29199834  0.46210832  0.00661321]
27627
[ 2.64572603 -0.25211856 -0.98671043 -0.35396032 -0.06287975  0.77188679
  0.2615988  -0.71642828  0.72767335 -0.06259902 -0.40002808 -1.49282505
 -0.77444957 -1.03803287 -0.22656516 -0.2186423  -2.2651272  -0.32119019
 -0.54109369  0.20243115 -0.04802043 -0.10931328  0.147

In [383]:
# synthetic=synthetic[0:synthetic.shape[0]-,:]
labels = np.ones(synthetic.shape[0]) #Kennzeichnung der neuen synthetischen samples als Min instance mit class=1
synthetic = np.insert(synthetic, synthetic.shape[1], values=labels, axis=1)
examples = np.concatenate((samples, synthetic), axis=0) #Verbindung der samples mit den synthetischen samples

In [ ]:
# def Euclidean_Metric(a,b):
#       dis=0
#       A = np.array(a)
#       B = np.array(b)
#       n=A.shape[0]
#       for i in range(n):
#           dis=dis+(A[i]-B[i])*(A[i]-B[i])
#       dis=np.sqrt(dis)
#       return dis

# # Algorithm 1: Noise filtering
# dis_matrix = np.zeros((Minority_X.shape[0], All_X.shape[0]), dtype=float) #leere Matrix (492x284807) nur trainingsdatenset

# for i in range(0, Minority_X.shape[0]):
#     for j in range(0, All_X.shape[0]):
#         dis_matrix[i,j] = Euclidean_Metric(Minority_X[i,:], All_X[j,:]) #berechne für jede Min instance die Eucl. distance zu allen sampels im Datensatz
#         if(dis_matrix[i,j] == 0): #Eucl. distance zu sich selbst
#             dis_matrix[i,j] = 999999 #Min instance darf nicht sein eigener nächster Nachbar sein
# dis_matrix = dis_matrix.tolist()

In [ ]:
# d=[] #list for noise Min instances
# #print(Minority_X.shape[0])
# for i in range(Minority_X.shape[0]): #für alle Min instances
#     min_index = list(map(dis_matrix[i].index, heapq.nsmallest(5, dis_matrix[i]))) #index des nächsten Nachbarn in Zeile der aktuellen Min instance
#     print(min_index)
#     if(samples_y[min_index[0]] == 0): #nearest neighbour ist aus Maj class -> Noise
#         d.append(i)
# Minority_X = np.delete(Minority_X, d, axis=0) #lösche alle Noise samples aus Min set
# #print(Minority_X.shape)
# n = int((n1)/Minority_X.shape[0])
# #print(n)

In [ ]:
# Algorithm 2: Adaptive neighbor instances selection
n = 10 #number of synthetic new instances per qualified Min instance
k = 5
synthetic = np.zeros(((Minority_X.shape[0])*n,Minority_X.shape[1]),dtype=float) #Matrix(qualified Min instances * n x Number of Features)

#print(Minority_X.shape[0])
for i in range(Minority_X.shape[0]):
    min_index=list(map(dis_matrix[i].index, heapq.nsmallest(k, dis_matrix[i]))) #indizes der k nächsten Nachbarn in Zeile der aktuellen qualif. Min instance
    best_index={}
    best_f=0
    for h in range(len(min_index)):
        
        if(samples_y[min_index[h]]==0): #if h. nearest neighbour belongs to Maj class set -> STOP
            best_index[best_f]=min_index[h]
            best_f+=1
            break
        else:
            best_index[best_f]=min_index[h] #else append h. nearest neighbour to set of qualified nearest neighbours for qualified Min instance
            best_f+=1
    #print(best_index)

    # Algorithm 3: Procedure of ASN-SMOTE (Create new synthetic minority samples)
    for j in range(0,n):
        nn=random.randint(0,len(best_index)-1) #Zufälliges Auswählen eines besten Nachbarn aus dem Set der qualifizierten Nachbarn
        #print(min_index[nn])
        dif=All_X[best_index[nn]]-Minority_X[i] #Differenz des zufällig ausgewählten Nachbarn und der Min instance
        #print(dif)
        gap=random.random() #Wert zw. 0 und 1
        synthetic[g_index]=Minority_X[i]+gap*dif #Generierung einer neuen Min instance
        g_index+=1

In [ ]:
samples_y[min_index]
best_index

In [ ]:
Minority_X.shape[0]

In [ ]:
samples_y[min_index]
#best_index

In [ ]:
#print(synthetic.shape)
#print(wrg)

# synthetic=synthetic[0:synthetic.shape[0]-,:]
labels=np.ones(synthetic.shape[0]) #Kennzeichnung der neuen synthetischen samples als Min instance mit class=1
synthetic=np.insert(synthetic,synthetic.shape[1],values=labels,axis=1)
examples=np.concatenate((samples,synthetic),axis=0) #Verbindung der samples mit den synthetischen samples
time_end=time.time()
del(dis_matrix)

In [ ]:
len(examples) - len(All_X)